In [2]:
!pip install pandas

    100% |████████████████████████████████| 10.1MB 656kB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 11.9MB/s ta 0:00:01
    100% |████████████████████████████████| 17.3MB 546kB/s eta 0:00:01


In [3]:
import pandas as pd
#import numpy as np
#import requests # library to handle requests
#import json # library to handle JSON files
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.6.13

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          85 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge


geopy-1.19.0         | 53 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ########################

In [13]:
min_date = "1+February+2019"
csv_link = "http://landregistry.data.gov.uk/app/ppd/ppd_data.csv?et%5B%5D=lrcommon%3Afreehold&et%5B%5D=lrcommon%3Aleasehold&header=true&limit=all&min_date={}&nb%5B%5D=true&nb%5B%5D=false&ptype%5B%5D=lrcommon%3Adetached&ptype%5B%5D=lrcommon%3Asemi-detached&ptype%5B%5D=lrcommon%3Aterraced&ptype%5B%5D=lrcommon%3Aflat-maisonette&ptype%5B%5D=lrcommon%3AotherPropertyType&tc%5B%5D=ppd%3AstandardPricePaidTransaction&tc%5B%5D=ppd%3AadditionalPricePaidTransaction".format(min_date)
df = pd.read_csv(csv_link, header=None)
#df = pd.read_csv("http://landregistry.data.gov.uk/app/ppd/ppd_data.csv?et%5B%5D=lrcommon%3Afreehold&et%5B%5D=lrcommon%3Aleasehold&header=true&limit=all&min_date=1+February+2019&nb%5B%5D=true&nb%5B%5D=false&ptype%5B%5D=lrcommon%3Adetached&ptype%5B%5D=lrcommon%3Asemi-detached&ptype%5B%5D=lrcommon%3Aterraced&ptype%5B%5D=lrcommon%3Aflat-maisonette&ptype%5B%5D=lrcommon%3AotherPropertyType&tc%5B%5D=ppd%3AstandardPricePaidTransaction&tc%5B%5D=ppd%3AadditionalPricePaidTransaction",  header=None)
print(df.shape)

(23674, 16)


In [14]:
headers = ['unique_id','price_paid','deed_date','postcode','property_type','new_build','estate_type','saon','paon','street','locality','town','district','county','transaction_category','linked_data_uri']
#df.rename(columns=headers)
df.columns = headers

#remove 'additional' transaction category
df = df[df['transaction_category']=='A']

df = df[df['property_type']=='F']

df = df[df['county']=='GREATER LONDON']
print(df.shape)

df.drop(columns=['unique_id','deed_date', 'property_type', 'new_build','estate_type','saon','paon','street','locality','town','district','county','transaction_category','linked_data_uri'], inplace=True)

df.head()

(830, 16)


,unique_id,price_paid,postcode
2334,8355F009-61A2-55C5-E053-6B04A8C0D090,240000,BR1 3AG
2335,8355F009-601F-55C5-E053-6B04A8C0D090,315000,BR1 3AX
2342,8355F009-6078-55C5-E053-6B04A8C0D090,280000,BR1 3TY
2353,8355F009-5F95-55C5-E053-6B04A8C0D090,200000,BR2 0AB
2354,8355F009-60B5-55C5-E053-6B04A8C0D090,280000,BR2 0JW


In [15]:
#Add lontitude and lattitude data
addresses = df['postcode']
latitude = []
longitude = []

for address in addresses:
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address+", UK", timeout=2)
    try:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append(0)
        longitude.append(0)
        
df['Latitude']= latitude
df['Longitude']= longitude

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests